# Импорт библиотек

In [159]:
import os
import pandas as pd
import glob
import string
import nltk
import re
import numpy as np

# Отсев невалидных данных и определение валидных


In [160]:
class Utils():
    root_directory = os.chdir('c:/Users/iZiPC.by/notebooks/parser/')# input path
    list_of_files = glob.glob('**/*.txt', recursive=True)
    invalid_parsing = []
    trash = ['Волковыск-1.txt','Волковыск.txt','Вороново-1.txt',
         'Вороново.txt', 'Черновик-1.txt', 'Черновик.txt']
    for file in list_of_files:
        try:
             with open(file, encoding='utf-8') as f:
                    num = f.readline()
                    file_ = f.read()
                    if num and file_:
                        pass
        except:
            invalid_parsing.append(file)
    for i in invalid_parsing:
        list_of_files.remove(i)
    for i in trash: 
        list_of_files.remove(i)


list_of_files = Utils.list_of_files

# Определяем пустые списки, которые будут наполняться данными


In [161]:
plt_ = []
wbc_ = []
hgb_ = []
crea_ = []
ldh_ = []
sex_ = []
crp_ = []
tr = []
bir = []
ad = []
dis = []
ID_ = []
dgs_ = []
rf_ = []
alt_ = []
ast_ = []
pct_ = []

# Методы, которые позволяют получить данные

In [162]:
def ID(num):
  """returns ID of the patient"""
  for i in num.split():
    for char in i.split():
        if char.isnumeric():
            try:
                return int(char)
            except:
                return 'None'
            
            
def treatment(file_):
  """ returns srting of remedies used for the patient """
  for line in file_.split('\n'):
    if re.search('Провед.нное.лечение', line):  # Does the same thing as "if 'hello' in line:"
        return(line.replace('Проведенное лечение:', \
                            '').replace('ЛФК', '').replace('ФТЛ', ''))
    

def born_adm_disch(file_):
    """ finds all nessesary dates in the epicrisis"""
    pattern = re.compile("(\d{2}).(\d{2}).(\d{4})") # check all dates
    birthday, admission = pattern.findall(file_)[:2]
    discharging = pattern.findall(file_)[-1]
    try:
        return ':'.join(birthday), ':'.join(admission), ':'.join(discharging)
    except:
        return 'None'


pattern_diagnosis = ['Диагноз:','Диагноз клинический:','Диагноз', 'Д-з:',
                     "Диагноз заключительный:"]
def diagnosis_dirty(file_):
  int_ = 0
  for i in file_.split():
    int_ += 1
    if i in pattern_diagnosis:
      return file_.split()[int_+1:int_+40]    
  

def crp(file_):
  """returns all crp values"""
  file_= ''.join(file_.split())
  file_ = re.sub(r'[ЦСC]РБ|\w\D(реактив.|реакт.|реак.)белок', 'С-реактивныйбелок', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'(?<!С-реактивныйбелок)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_)
  pattern_1 = re.compile(r'(?:\w\Dреактивныйбелок|\w\Dреактивныйбелокдо)(\d*\.\d+|\d+)')
  pattern_2 = pattern_1.findall(file_)
  try:
    if pattern_2:
      return np.float_(pattern_2)
    else:
      return 'None'
  except:
    pass


def gender(file_):
  file_ = file_.title()
  file_ = ''.join(file_.split())
  file_ = re.sub(r'Диагноз.*', '', file_)
  patt1 = re.compile(r'[А-Я](\w{,19}(\w|\ич))[А-Я]\w{,20}(вна)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return 'female'
    else:
      return 'male'
  except:
    pass
  

def ldh(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wактатдегидрогеназ\w', 'лдг', file_)
  file_ = re.sub(r'(?<!лдг)\d\d\.\d\d', '', file_)
  patt1 = re.compile(r'(?<=лдг)\d{,4}')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass



def cre(file_):
  """Returns all creatinine values"""
  file_= ''.join(file_.split())  
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'(\wреатини\w|креатин|креат)(?![а-яА-Я,])', 'cre', file_)
  file_ = re.sub(r'(?<!cre)\d\d\.\d\d', '', file_)
  patt1 = re.compile(r'(?<=cre)\d{2,3}')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass


def hgb(file_):
  """returns list of str with level of hemoglobin"""
  file_= ''.join(file_.split())  
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  patt = re.compile(r'гемоглобин|гемогл|гб|гем')
  patt1 = re.sub(patt, r'hgb', file_)
  patt2 = re.compile(r'(?<=hgb)\d{2,3}')
  all_hgb = patt2.findall(patt1)
  try:
    if all_hgb:
      return np.float_(all_hgb)
    else:
      return 'None'
  except:
    pass


def wbc(file_):
  """returns list of str with level of wbc"""
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\Dбщийанализкрови', 'оак', file_)
  file_ = re.sub(r'(?<=оак)гб\d*,', '', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=оак|wbc)(?:л|лейкоцит\w)(\d*\.\d+|\d+)')
  all_wbc = patt1.findall(file_)
  try:
    if all_wbc:
      return np.float_(all_wbc)
    else:
      return 'None'
  except:
    pass


def plt(file_):
  """returns list of str with level of plt"""
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  patt1 = re.compile(r'\wромбоцит\w|(?<![а-яА-Я])тр(?![а-яА-Я])')
  file_ = re.sub(patt1, r'plt', file_)
  file_ = re.sub(r'(?<!plt)\d\d\.\d\d', '', file_)
  patt2 = re.compile(r'(?<=plt)\d{3}')
  all_plt = patt2.findall(file_)
  try:
    if all_plt:
      return np.float_(all_plt)
    else:
      return 'None'
  except:
    pass


def rf(file_):
  file_= ''.join(file_.split())  
  file_ = re.sub(r'[а-яА-ЯёЁ]ДН[^Шш0-9]', 'ДН'.lower(), file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{3}|[Шш]', '3', file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{2}|1-2', '2', file_)
  file_ = re.sub(r'(?<=ДН)[оoОO]', '0', file_)
  patt1 = re.compile(r'(?<=ДН)\d')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 0
  except:
      return 0



def alt(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wланинаминотрансфераз\w|алат', 'алт', file_)
  file_ = re.sub(r'(?<!алт)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_) 
  patt1 = re.compile(r'(?<=алт)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass


def ast(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wспартатаминотрансфераз\w|асат', 'аст', file_)
  file_ = re.sub(r'(?<!аст)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=аст)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass


def pct(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'(?<=\,\d)\,|\.\d{2}\.\d\d\.\d{2,4}.', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}(\.|)', '', file_)
  file_ = re.sub(r'[–-]|[():<>=]', '', file_)
  file_ = re.sub(r'менее', '', file_)
  file_ = re.sub(r'от', '', file_)
  file_ = re.sub(r'(\wрокальцитони\w|прокальцитон|прокальцит|прокальц|рст)(?![а-яА-Я,])', 'pct', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=pct)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass

# Запускаем методы

In [163]:
for file in list_of_files:
    try:
         with open(file, encoding='utf-8') as f:
                num = f.readline()
                file_ = f.read()
                ID_.append(ID((num)))
                tr.append(treatment((file_)))
                bir.append(born_adm_disch(file_)[0])
                ad.append(born_adm_disch(file_)[1])
                dis.append(born_adm_disch(file_)[2])
                dgs_.append(diagnosis_dirty((file_)))
                crp_.append(crp(file_))
                sex_.append(gender(file_))
                ldh_.append(ldh(file_))
                crea_.append(cre(file_))
                hgb_.append(hgb(file_))
                wbc_.append(wbc(file_))
                plt_.append(plt(file_))
                rf_.append(rf(file_))
                alt_.append(alt(file_))
                ast_.append(ast(file_))
                pct_.append(pct(file_))
    except:
        print('stp')

# !!! Работа со списками
## 1) Начнём с диагнозов

In [164]:
diagnosis_cleaned =  []
for diagnos in dgs_:
    try:
        diagnosis_cleaned.append(' '.join(diagnos))
    except:
        diagnosis_cleaned.append('None')
        
        
        
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))





list_wo_punct = []
for sent in diagnosis_cleaned:
    try:
        sent = remove_punctuation(sent.lower())
        list_wo_punct.append(sent)
    except:
        pass
        


Таким образом, мы имеет список со строками без пунктуции, далее, нам надо найти ключевые слова, чтобы определить наличие или отсутствие конкретных заболеваний

# # - поиск коронавирусной инфекции. 
- если есть коронавирусная инфекция - 1, иначе - 0

In [165]:
covid_list = []    
for sent in list_wo_punct:
    pattern = re.sub(r'covid19', 'covid', sent )
    pattern = re.sub(r'коронавирусная', 'covid', pattern)
    pattern = re.sub(r'короновирусная', 'covid', pattern)
    pattern = re.sub(r'b342', 'covid', pattern)
    pattern = re.sub(r'sarscov2', 'covid', pattern)
    pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'covid')
    patt_2 = re.search(pattern_1, pattern)
    try:
        covid_list.append(patt_2)
    except:
        pass
    
    
c =[]
#если есть коронавирусная инфекция - 1, иначе - 0
for i in covid_list:
    if not i:
        c.append(0)
    else:
        c.append(1) 

# # - поиск пневмоний

In [166]:
pn_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'пневмония', 'J18', sent )
    pattern = re.sub(r'пне...ния', 'J18', pattern)
    pattern = re.sub(r'внегоспитальная', 'J18', pattern)
    pattern = re.sub(r'внебольничная', 'J18', pattern)
    pattern_1 = re.compile(r'J18')
    patt_2 = re.search(pattern_1, pattern)
    try:
        pn_list.append(patt_2)
    except:
        pass  
    
    
p = []
# если есть пневмония - 1, иначе - 0
for i in pn_list:
    if not i:
        p.append(0)
    else:
        p.append(1)

## Поиск ишемической болезни сердца

In [167]:
ihd_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'ссн', 'ИБС', sent )
    pattern = re.sub(r'ибс', 'ИБС', pattern)
    pattern = re.sub(r'атеросклеротический', 'ИБС', pattern)
    pattern = re.sub(r'кардиосклероз', 'ИБС', pattern)
    pattern = re.sub(r'фп', 'ИБС', pattern)
    pattern = re.sub(r'фибрил....', 'ИБС', pattern)
    pattern_1 = re.compile(r'ИБС')
    patt_2 = re.search(pattern_1, pattern)
    try:
        ihd_list.append(patt_2)
    except:
        pass  
# итого, обработаны все возможные паттерны    
    
ihd = []

for i in ihd_list:
    if not i:
        ihd.append(0)
    else:
        ihd.append(1)

d = {}
for i in ihd:
    d[i]=d.get(i, 0)+1
d

{0: 1247, 1: 977}

Итого мы имеет 977 пациентов с ИБС. 

## Поиск сахарного диабета

In [168]:
dm_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'сд', 'СД', sent )
    pattern = re.sub(r'сах.....', 'СД', pattern)
    pattern = re.sub(r'д.абет', 'СД', pattern)
    pattern_1 = re.compile(r'СД')
    patt_2 = re.search(pattern_1, pattern)
    try:
        dm_list.append(patt_2)
    except:
        pass  
    
dm_ = []

for i in dm_list:
    if not i:
        dm_.append(0)
    else:
        dm_.append(1)
        
d = {}
for i in dm_:
    d[i]=d.get(i, 0)+1
d

{0: 2096, 1: 128}

128 пациентов с сахарным диабетом

## Поиск ожирения

In [169]:
obes_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'ожир.н..', 'ОЖИРЕНИЕ', sent )
    pattern = re.sub(r'.нжо.', 'ОЖИРЕНИЕ', pattern)
    
    #pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'ОЖИРЕНИЕ')
    patt_2 = re.search(pattern_1, pattern)
    #patt_2 = pattern_1.findall(''.join(pattern.split()))
    try:
        obes_list.append(patt_2)
    except:
        pass  
    
ob_ = []

for i in obes_list:
    if not i:
        ob_.append(0)
    else:
        ob_.append(1)
    
d = {}
for i in ob_:
    d[i]=d.get(i, 0)+1
d

{0: 2115, 1: 109}

 109 пациентов с ожирением

## Поиск обструктивной болезни сердца

In [170]:
copd_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'хобл', 'ХОБЛ', sent )
    pattern = re.sub(r'другая обструктивная', 'ХОБЛ', pattern)
    pattern = re.sub(r'j44', 'ХОБЛ', pattern)
    pattern_1 = re.compile(r'ХОБЛ')
    patt_2 = re.search(pattern_1, pattern)
    try:
        copd_list.append(patt_2)
    except:
        pass  


copd_ = []
for i in copd_list:
    if not i:
        copd_.append(0)
    else:
        copd_.append(1)    
        
d = {}
for i in copd_:
    d[i] = d.get(i, 0) + 1
d

{0: 1833, 1: 391}

391 пациент с хронической обструктивной болезнью лёгких

## Поиск бронхиальной астмы

In [171]:
ba_list = []
for sent in list_wo_punct:
  pattern = re.sub(r'бронхиальная', 'БА', sent )
  pattern = re.sub(r'частично контролируемая', 'БА', pattern)
  pattern = re.sub(r'\sба\s', 'БА', pattern )
  pattern = re.sub(r'неконтролируемая', 'БА', pattern )
  pattern_1 = re.compile(r'БА')
  pattern_2 = re.search(pattern, sent)
  try:
    ba_list.append(pattern_2)
  except:
    pass
ba = []
for i in ba_list:
    if not i:
        ba.append(0)
    else:
        ba.append(1)

d = {}
for i in ba:
    d[i] = d.get(i, 0) + 1
d

{1: 2065, 0: 159}

159 пациентов с бронхиальной астмой

## Определяем наличие дыхательной недостаточности у пациентов

In [172]:
# нас интересует самый высокий уроведь ДН, т.к. у пациента при поступлении ДН может быть 2, а при выписке - 0,
# о тяжести пациента мы будем судить по наибольшему значению
rfl = []
for i in rf_:
    if i:
        rfl.append(max(np.float_(i)))
    else:
        rfl.append(0)

d = {}
for i in rfl:
    d[i] = d.get(i, 0) + 1
d

{0.0: 1015, 1.0: 652, 2.0: 535, 3.0: 22}

итого: по степеням дыхательной недостаточности мы имеем: {0.0: 1015, 1.0: 652, 2.0: 535, 3.0: 22}

## Определение тяжести

Пневмонии классифицируются на тяжёлые и нетяжёлые

In [173]:
gr_list = []
for sent in list_wo_punct:
  pattern = re.sub(r'среднетяж.л..', '', sent)
  pattern = re.sub(r'ср.тяж.', '', sent)
  pattern = re.sub(r'нетяж.л..', '', pattern)
  pattern_1 = re.compile(r'тяж.л..')
  pattern_2 = re.search(pattern_1, pattern)
  try:
    gr_list.append(pattern_2)
  except:
    pass

In [174]:
g = []
for i in gr_list:
    if not i:
        g.append(0)
    else:
        g.append(1)
d = {}
for i in g:
    d[i] = d.get(i, 0) + 1
d

{0: 2016, 1: 208}

Итого у нас 208 тяжёлых пневмоний

## 2) Работа с лечением. 

In [175]:
# задаём словарь, в котором ключ - истинное название препарата, а значения - варианты названий ( коммерческих) и
# названия с ошибками для обработки с помощью регулярных выражений

dict_treatment = {'лизиноприл':'л.з.....ил|лизитар|лизинеоприл', 
                  'цефтриаксон':'три.....ф|цефт.....он|цефтриакосн', 
                   "метопролол":"мет.....ол|эгилок", 
                  "фраксипарин":"фра......ин",
                  'тейкопланин': "тей......ин.",
                  "амброксол": "аброкол|амброко.|амб......|амброксолс|амброкосл|амброксолл|амброксолл|аброксол|амброксолл", 
                  "моксифлоксацин": 'пле....кс.|мокс.......цин',
                  "молсидомин":"м.лсид...н",
                  "спиронолактон":"сп.р.......он.|верош....н",
                 "бисопролол":"б.с.пр..ол|бисопралдол|бикард|конкор", 
                  "лансопрозол": "лан......ол|ланс.зол",
                 "меропенем":"мер....ем",
                  "аторвастатин": "липромак|аторв.......",
                 "дексаметазон": "декс.......н|дексаетазон",
                 "левофлоксацин": "лев.......цин",
                 "омепразол": "омез|омепр..ол",
                 "азитромицин": "азит.......",
                 "кандесартан": "к.с.рк",
                 "амоксициллин": "амоксицилин|амо........н",
                 "озельтамивир": "флустоп|озел.......р",
                 "эноксапарин": "клексан|кл...ан",
                 "имепенем": "им....ем",
                 "клопидогрел": "клопидогрель|плавикс|клорпидогрель",
                 "аспикард": "аспкиард|кардиомагнил|ас....рд",
                 "амлодипин":"амл....ин",
                 "амикацин": "ам.к...н",
                 "индапамид": "индап|инд.....д",
                 "беродуал": "бер.дуал|ипратропия.бромид",
                 "ацетилцистеин": "ацецемед|асс|ацц|ацемед|ацецезон",
                 "парацетамол": "парац.....л",
                 "гидроксихлорохин": "плаквенил|имар.|иммар.",
                 "метилпреднизолон": "медрол|м.дрол|метипред|диметилпреднизолон",
                 "теофиллин": "теофилин|теотард|теофилл",
                 "кларитромицин": "кларибацин|кла.......цин",
                 "лозартан": "лазртан|л...ртан|л.риста",
                 "валсартан": "валозартан|валз|вал....ан",
                 "моксонидин": "м.кс.н.дин",
                 "амиодарон": "ами.д.р.н|к.рд.рон",
                 "полимиксин": "колистин|колистат|к.л.стат",
                 "преднизолон": "преднезолон|пре.н.з.лон",
                 "рамиприл": "рам.л...|рам.прил",
                 "карбомазепин": "карб.м.з.п.н",
                 "кетаролак": "кет.р.лак|кет.рол|кет.н.в",
                 "гентамицин": "г.нт.мицин",
                 "цефепим": "ц.ф.пим",
                 "цефоперазон/сульбактам": "цеф.пер.зон|цеф.пер.зон.су.......",
                 "тайгециклин": "тиг.циклин|та.г.циклин",
                 "зопиклон": "з.п.клон|соне.|соне.с"}

cleaned_treatment = []
for words in tr:
    try:
        for i, j in dict_treatment.items():
            words = re.sub(j, i, words.lower())

        cleaned_treatment.append(words)
    except:
        cleaned_treatment.append("None")
        
        
        

На выходе мы получили список с правильными названиями препаратов. Если нет - возвращаем "Ноне"

# 3) Работа с анализами

Мотивировочная часть:
Прокальцитонин - показатель, который обладает довольно выской чувствительностью и специфичностью к наличию у пациента батериальной инфекции, т.о., если уровень прокальцитонина в пределах референтных значений - антибиотики пациенту не показаны. 
Таким образом, мы хотим знать уровень прокальцитонина у пациента при поступлении и максимальный прокальцитонин (нужны ли были ему антибиотики за всё время пребывания в больнице)

In [153]:
# при первичной оценке данных, оказалось, что имеет место уровень прокальцитонина 7.0, что невозможно. Это просто неправильный
# парсинг. Поэтому заменим на 0.07


In [176]:
initial_pct = []
max_pct = []
try:
    for pct in pct_:
        if pct != 'None':
            initial_pct.append(pct[0])
            max_pct.append(max(pct))
        else:
            initial_pct.append(0)
            max_pct.append(0)
            
except:
    initial_pct.append(0)
    max_pct.append(0)

<ipython-input-176-996c0f4efaac>:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pct != 'None':


In [177]:
max(initial_pct)

7.0

In [179]:
initial_pct_ = list(map(lambda x: x / 100 if x > 5 else x, initial_pct))
max_pct_ = list(map(lambda x: x / 100 if x > 5 else x, max_pct))


In [181]:
max(initial_pct_) 

0.98

## Показанием для антибиотикотерапии является уровень прокальцитонина более 0.5. Определим этот признак

In [188]:
ind_4_ab = []
for pct_i  in  max_pct_:
    if pct_i > .49:
        ind_4_ab.append(1)
    else:
        ind_4_ab.append(0)

In [189]:
len(ind_4_ab)

2224

Отлично, проблема решена

## Поработаем с гемоглобином. В целом, нас будет интересовать наибольший уровень, чтобы определить наличие анемии 

In [207]:
hgb__ = []
try:
    for hgb in hgb_:
        if max(hgb) > 120:
            hgb__.append(0)
        else:
            hgb__.append(1)
                       
except:
    hgb__.append(0)
    

In [208]:
len(hgb__)

14